In [2]:
from openai import OpenAI
import jsonlines
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())
 
OPEN_AI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Leer el archivo JSONL
input_file = 'wnli/wnli-dev-es.jsonl'
results = []

with jsonlines.open(input_file) as reader:
    for obj in reader:
        prompt = obj['prompt']
        true_label = obj['referenceResponse']

        client = OpenAI(api_key=OPEN_AI_API_KEY)

        response = client.chat.completions.create(
            model="gpt-4o",
            messages= [{
                    "role": "user", 
                    "content": "{}".format(prompt)
                }],
        )

        predicted_label = response.choices[0].message.content

        # Guardar el prompt, la categoría verdadera y la categoría predicha
        results.append({
            'prompt': prompt,
            'true_label': true_label,
            'predicted_label': predicted_label
        })

# Convertir resultados a DataFrame
df_results = pd.DataFrame(results)

# Guardar los resultados en un archivo CSV
df_results.to_csv('evaluation_resultsgpt4o.csv', index=False)